# Excercise 5.12 : Estimate significant figures

In [1]:
from math import pi, exp, log

## Task 
Calculate the StefanBoltzmann constant,  
$\sigma = \frac{k_B^4}{4\pi^2c^2\hbar^3}I$ with $I=\int_0^\infty dx\frac{x^3}{e^x-1}$.  
You must be confident that your calculated result matches the true value to three significant figures.

### True value
In fact, $I$ can be calculated analytically using the Riemann function and zeta function (horribly chatGPT knows that!).  
The result is $I=\frac{\pi^4}{15}$. 

In [2]:
print(pi**4/15)

6.493939402266828


### Method 
#### Integral approximation  
We use the trapezoid approximation with Euler-Maclaurin correction $h^2\frac{1}{12}(f'(a)-f'(b))$. The error of this integral approximation should be $\mathcal O(h^4)$.  
#### Transformation function to make integral domain finite  
We use $x=z/(1-z)$.

## Calculation

$I=\int_0^1 dz~g(z)$ with $g(z)=\frac{z^3}{(1-z)^5}\frac{1}{\exp(\frac{z}{1-z})-1}$.  
After tedious calculation, it turns out that $g'(0)=g'(1)=0$.  So the Euler-Maclaurin term vanishes;  the trapezoid approximation alone has accuracy up to $\mathcal O(h^4)$.  

### Estimation of numerical error
Let $I_{\rm tra}$ be the numerical result of the integral $I$ obtained by the trapezoidal method.  According to the analysis above, 
${\rm error} \equiv I_{\rm tra}-I = ch^4$ with some constant $c$.  
Now, suppose we halve the integration interval $h$, and let $\tilde I_{\rm tra}$ be the resulting value computed by the trapezoidal method.  Then $\tilde I_{\rm tra}-I = \frac{1}{16}ch^4$.  Then $I_{\rm tra} - \tilde I_{\rm tra}=\frac{15}{16}ch^4$.  

When $I_{\rm tra}$ matches $I$ to three significant figures, it means that $|{\rm error}/I_{\rm tra}|<10^{-4}$.  In other words, $|(I_{\rm tra}-\tilde I_{\rm tra})/I_{\rm tra}|<10^{-4}$ (here we omit the factor $\frac{15}{16}$ because it is almost equal to one.)  

Therefore, the strategy is that we gradually make $h$ small, and find $h$ at which $|(I_{\rm tra}-\tilde I_{\rm tra})/I_{\rm tra}|<10^{-4}$ holds.  

In [3]:
def trapezoid(f, a, b, N):
    "returns integral value of f from a to b using trapezoid approximation"
    assert a<b
    h = (b-a)/N
    result = 0.
    for k in range(1,N+1):
        xk = a + h*k
        result += h*f(xk)
    result += 0.5*h*(f(a) - f(b))
    return result

def calc_relative_error(f, a, b, N):
    "returns |\tilde I_{\tra} - I_{\tra}|/|I_{\rm tra}|"
    I  = trapezoid(f, a, b, N)
    tI = trapezoid(f, a, b, 2*N)
    return abs(I-tI)/abs(I)

def g(z):
    assert 0 <= z <= 1.
    eps = 1e-5
    if eps<z<1-eps:
        return (z**3/(1-z)**5) * (1 / (exp(z/(1-z)) - 1) )
    else:
        return 0
        
THRESHOLD = 1e-4
for N in range(10,30):
    relative_error = calc_relative_error(g, 0, 1, N)
    print(f"N={N}, relative_error={relative_error}")
    if relative_error < THRESHOLD:
        break
final_value = trapezoid(g, 0, 1, N)
print("final integral value:", final_value)

N=10, relative_error=0.0208641512797546
N=11, relative_error=0.020341736881876026
N=12, relative_error=0.01423753595725406
N=13, relative_error=0.007619031672207271
N=14, relative_error=0.0026101988379132863
N=15, relative_error=0.0003975219348575649
N=16, relative_error=0.0017691396930425752
N=17, relative_error=0.002065967483639459
N=18, relative_error=0.0017883691312036928
N=19, relative_error=0.0012920087593331084
N=20, relative_error=0.0007887305591647458
N=21, relative_error=0.0003799936873597221
N=22, relative_error=9.570734322286653e-05
final integral value: 6.494556552221367


Comparing this value to the true value $I=\frac{\pi^4}{15}=6.49393940226682$, we see that $I_{\rm tra}$ matches $I$ to three significant figures.  

Finally, we calculate $\sigma = \frac{k_B^4}{4\pi^2c^2\hbar^3}I_{\rm tra}$ with $I_{\rm tra}=6.494$ (here we keep $I_{\rm tra}$ to four digits because it is correct only to three significant figures).  

In [4]:
kB = 8.617333262 * 1e-5  # eV / K
chbar = 197.3269804 * 1e-9 # eV m
hbar = 6.582119569 * 1e-16 # eV s
# sigma has unit eV/(K^4 m^2 s)
sigma = kB**4/(4 * pi**2 * chbar**2 * hbar) * 6.494 * 1.60218e-19  # last factor is to convert eV to Joule
print(sigma)

5.670439248299456e-08


According to Wikipedia, the Stefan–Boltzmann constant is 5.670374419e-8 [J/(K^4 m^2 s)]

# Side task  
For $I=\int_0^1 dz~g(z)$ with $g(z)=\frac{z^3}{(1-z)^5}\frac{1}{\exp(\frac{z}{1-z})-1}$, the error of the trapezoidal approximation to the true value should be $\mathcal O(h^4)$ (since $g'(0)=g'(1)=0$ and the Euler-Maclaurin term vanishes).  
Verify this expectation.  

### Verification    
Let $I_{\rm tra}(h)-I = ch^\alpha$.  Our task is to see the value of $\alpha$.  
Since $I_{\rm tra}(h/2)-I = \frac{1}{2^\alpha}ch^\alpha$, we get $d(h) \equiv I_{\rm tra}(h)-I_{\rm tra}(h/2) = (1-\frac{1}{2^\alpha})ch^\alpha$.   
Since $d(h/2) = \frac{1}{2^\alpha}(1-\frac{1}{2^\alpha})ch^\alpha$, we get $d(h)/d(h/2) = 2^\alpha$.  

In [5]:
def trapezoid(f, a, b, N):
    "returns integral value of f from a to b using trapezoid approximation"
    assert a<b
    h = (b-a)/N
    result = 0.
    for k in range(1,N+1):
        xk = a + h*k
        result += h*f(xk)
    result += 0.5*h*(f(a) - f(b))
    return result

def g(z):
    assert 0 <= z <= 1.
    eps = 1e-5
    if eps<z<1-eps:
        return (z**3/(1-z)**5) * (1 / (exp(z/(1-z)) - 1) )
    else:
        return 0

def calc_alpha(N):
    d  = trapezoid(g, 0, 1, N) - trapezoid(g, 0, 1, 2*N)
    d2 = trapezoid(g, 0, 1, 2*N) - trapezoid(g, 0, 1, 4*N)
    #print(d, d2, d/d2)
    alpha = log(abs(d/d2), 2)
    return alpha

for N in range(50,150):
    print(N, calc_alpha(N))

50 12.972285281895125
51 13.079160083351235
52 12.949157282897822
53 12.413217761992566
54 11.174908435943047
55 7.782803201556371
56 10.786795588595565
57 11.3142170777595
58 11.474805785707655
59 11.483020193769663
60 11.404448554290578
61 11.263012809875674
62 11.065651843690194
63 10.810319979611599
64 10.4881013482088
65 10.080491121737968
66 9.551916660009542
67 8.8206200737143
68 7.616200092368787
69 3.607667449893733
70 7.424485005788675
71 8.152834871768567
72 8.472157028456133
73 8.604956750790105
74 8.622624947556428
75 8.555950291655138
76 8.419630058623758
77 8.219585406476284
78 7.955199616247061
79 7.618836577449121
80 7.1935877053534325
81 6.640537281823625
82 5.863157104075609
83 4.495848917700224
84 2.77164811604478
85 4.907253088681647
86 5.564460003849378
87 5.889797187733371
88 6.058057854627107
89 6.129776157132241
90 6.134455553870352
91 6.087970093671214
92 6.000027693775015
93 5.877770201229615
94 5.724724363430323
95 5.545407939833012
96 5.342328270962222
97 5

As expected, $\alpha$ goes to $4$ as we decrease $h$.  

But note that $\alpha$ deviates much from its theoretical value $4$ when $N<100$.  
I do not find a clear reason for this behavior.  One guess is as follows :  
Our analysis assumes that $I_{\rm tra}(h)-I$ can be well approximated as $I_{\rm tra}(h)-I = c_4h^4$, that is, the higher order terms $\mathcal O(h^6)$ can be neglected. But when the coefficient $c_6$ in the expansion $I_{\rm tra}(h)-I = c_4h^4 + c_6 h^6 + \cdots$, is large, our assumption is not valid  when $h$ is not sufficiently small.  So one possibility is that $c_6$ is large.  

# TO DO :
How accuracy will the numerical results be if we use transformation $x=\tan(z)$ instead of $x=z/(1-z)$ ?